# Setting up paths

In [ ]:
# Importing the os module for operating system-related functionality
import os

In [ ]:
# Custom model name for the project
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

# Pretrained model details
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

# Script for generating TFRecord
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

# Label map file name
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
# Dictionary containing various paths used in the project
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [ ]:
# Dictionary containing file paths
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# Creating directories if they don't exist
for path in paths.values():
    if not os.path.exists(path):
        !mkdir {path}

# Cloning Tensorflow Pretrained models ( TF Model Zoo )

In [ ]:
# utility module for downloading files from the internet.
!pip install wget

In [ ]:
import wget

In [ ]:
# Check if the directory containing TensorFlow object detection code does not exist
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    
    # Clone the TensorFlow models repository from GitHub into the specified path
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}


In [ ]:
# URL for downloading the Protocol Buffers compiler zip file
url = "https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"

# Download the Protocol Buffers compiler zip file
wget.download(url)

# Move the downloaded file to the specified PROTOC_PATH
!move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}

# Navigate to the PROTOC_PATH and extract the contents of the zip file
!cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip

# Add the bin directory of the Protocol Buffers compiler to the PATH environment variable
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))

# Navigate to the TensorFlow models research directory and compile Protocol Buffers proto files
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=.

# Copy setup.py file from object_detection/packages/tf2 to the research directory
# Build and install the object_detection package
!cd Tensorflow/models/research && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install

# Navigate to the slim directory and install the slim package
!cd Tensorflow/models/research/slim && pip install -e .


In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
# installing modules absent while verifing files ( missing modules are variable with each installation )

!pip install tensorflow-gpu==2.10.0
!pip install tensorflow==2.10.0
!pip install pyyaml
!pip install matplotlib
!pip install protobuf==3.20.0
!pip install pytz
!pip install tzdata==2022.7

In [ ]:
# Download the pre-trained model from the specified URL
wget.download(PRETRAINED_MODEL_URL)

# Move the downloaded tar.gz file to the PRETRAINED_MODEL_PATH
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}

# Navigate to the PRETRAINED_MODEL_PATH and extract the contents of the tar.gz file
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}


# Creating Label Map

In [ ]:
# List of labels, each with a name and an ID
labels = [{'name':'Face', 'id':1},{'name':'ThumbsUp', 'id':2},{'name':'ThumbsDown', 'id':3},
          {'name':'Hello', 'id':4},{'name':'ThankYou', 'id':5}, {'name':'Yes', 'id':6},
          {'name':'No', 'id':7}, {'name':'LiveLong', 'id':8}]

# Open the label map file for writing
with open(files['LABELMAP'], 'w') as f:
    # Iterate over each label and write the corresponding entries in the label map
    for label in labels:
        f.write('item { \n')               # Write the start of an item entry
        f.write('\tname:\'{}\'\n'.format(label['name']))  # Write the class name
        f.write('\tid:{}\n'.format(label['id']))          # Write the class ID
        f.write('}\n')                    # Write the end of the item entry


# Creating Tensorflow records

In [ ]:
# Check if the file generate_tfrecord.py does not exist
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    
    # Clone the specified GitHub repository into the SCRIPTS_PATH
    !git clone https://github.com/ankursinghbisht/GenerateTFRecord {paths['SCRIPTS_PATH']} 

In [ ]:
# Run the generate_tfrecord.py script for training data
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}
# Run the generate_tfrecord.py script for testing data
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}

# Copying Model Config to Training Folder

In [ ]:
# Copy the 'pipeline.config' file from the pretrained model directory to the checkpoint directory
!copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Updating Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# Load the existing pipeline configuration from the file
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

In [ ]:
# Read the content of the pipeline configuration file and merge it with the protobuf message
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
# Modify specific parameters in the pipeline configuration for transfer learning
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]


In [ ]:
# Convert the modified pipeline configuration back to a text format
config_text = text_format.MessageToString(pipeline_config)

# Write the updated configuration back to the pipeline configuration file
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:
    f.write(config_text)